# SGLang部署ERNIE-4.5-0.3B模型教程

## SGLang简介

SGLang（Structured Generation Language）是由UC Berkeley和Stanford联合开发的高性能大语言模型服务框架。该框架通过协同设计后端运行时和前端编程语言，实现了比传统框架更快的推理速度和更灵活的控制能力。SGLang在学术界和工业界都获得了广泛认可，被多家知名公司采用用于生产环境。

| 特性类别 | 具体功能 | 技术优势 |
|---------|---------|----------|
| **后端运行时优化** | RadixAttention前缀缓存 | 通过共享前缀减少重复计算，提升批处理效率 |
| | 零开销CPU调度器 | 消除调度延迟，提高GPU利用率 |
| | 预填充-解码分离 | 分离计算密集和内存密集操作，优化资源利用 |
| | 推测解码 | 并行生成多个候选token，加速生成过程 |
| | 连续批处理 | 动态批处理管理，提高吞吐量 |
| **前端编程接口** | 结构化生成 | 支持JSON、正则表达式等结构化输出 |
| | 链式调用 | 简化复杂对话和多轮交互的编程 |
| | 并行执行 | 原生支持并发请求处理 |
| **模型支持** | 生成模型 | ERNIE、Qwen、DeepSeek等主流模型 |
| | 量化支持 | FP4/FP8/INT4/AWQ/GPTQ等量化方法 |

## ERNIE-4.5模型支持

SGLang原生支持ERNIE-4.5系列模型，包括：
- **ERNIE-4.5-0.3B** - 轻量级密集模型
- **ERNIE-4.5-21B-A3B** - MoE模型
- **ERNIE-4.5-300B-A47B** - 大型MoE模型


## 环境准备

### 安装SGLang

```bash
git clone -b v0.5.0rc2 https://github.com/sgl-project/sglang.git
cd sglang

pip install --upgrade pip
pip install -e "python[all]"
```bash

## 部署方式对比

| 部署方式 | 适用场景 | 优势 | 劣势 |
|---------|---------|------|------|
| **命令行服务** | 生产部署、API服务 | 高性能、稳定性好、支持并发 | 需要服务器管理 |
| **Python脚本** | 批量处理、自动化任务 | 灵活性高、易于集成 | 需要编程基础 |



## 部署ERNIE-4.5-0.3B模型

### 1. 环境准备和安装

In [1]:
# 验证安装
import sglang as sgl
import torch
print(f"SGLang版本: {sgl.__version__}")
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU设备: {torch.cuda.get_device_name()}")

SGLang版本: 0.5.0rc2
PyTorch版本: 2.8.0+cu128
CUDA可用: True
GPU设备: NVIDIA GeForce RTX 4090


### 2. 启动本地SGLang服务

我们以`baidu/ERNIE-4.5-0.3B-PT`为例，在终端启动进行部署

```bash
python -m sglang.launch_server \
    --model-path baidu/ERNIE-4.5-0.3B-PT \
    --host 0.0.0.0 \
    --port 30000 \
    --trust-remote-code
```

**注意事项**：
- 使用 `--host 0.0.0.0` 允许从Windows主机访问WSL中的服务
- 首次运行会自动从HuggingFace下载模型，配置镜像后下载速度会显著提升
- 服务启动后会显示类似 "Launched server at http://0.0.0.0:30000" 的信息

In [32]:
import requests
model_info = requests.get("http://127.0.0.1:30000/get_model_info")
if model_info.status_code == 200:
    info = model_info.json()
    print(f"\n📋 模型信息:")
    print(f"模型路径: {info.get('model_path', 'N/A')}")


📋 模型信息:
模型路径: baidu/ERNIE-4.5-0.3B-PT


### 3. 使用OpenAI兼容API调用

In [30]:
import openai

# 配置客户端
client = openai.OpenAI(
    base_url="http://localhost:30000/v1",
    api_key="EMPTY"  # SGLang不需要API密钥
)

# 发送聊天请求
response = client.chat.completions.create(
    model="baidu/ERNIE-4.5-0.3B-PT",
    messages=[
        {"role": "user", "content": "你好，请介绍一下你自己"}
    ],
    max_tokens=100,
    temperature=0.3
)

print(f"模型回复: {response.choices[0].message.content}")

模型回复: 你好呀！我是AI助手，很高兴能为你提供帮助。有什么我可以帮你的吗？比如回答问题、解答问题，或者只是聊聊日常话题？😊


### 4. 使用SGLang原生API

In [27]:
import sglang as sgl

# 设置后端
sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))

# 定义生成函数
@sgl.function
def chat_with_ernie(s, user_message):
    s += sgl.user(user_message)
    s += sgl.assistant(sgl.gen("response", max_tokens=100))

# 调用模型
state = chat_with_ernie.run(user_message="介绍下北京！")
print(f"ERNIE回复: {state['response']}")

ERNIE回复:  欢迎来到北京！北京作为中国的首都，有许多独特而热闹的地方呢。（手势示意）在这里，您可以参观著名景点，如故宫博物院、天坛公园、北海公园等，既能看到历史遗迹，又能感受北京的古韵今风。


## 总结

本教程展示了如何部署和使用ERNIE系列模型（以0.3B为例）。SGLang作为新一代LLM服务框架，在性能和易用性方面都有显著优势。通过本教程，您可以快速上手SGLang，并在自己的项目中应用ERNIE模型的强大能力。

### 下一步建议

| 学习方向 | 推荐资源 | 应用场景 |
|---------|---------|----------|
| **高级功能** | [SGLang官方文档](https://docs.sglang.ai/) | 生产环境部署、性能优化 |
| **模型微调** | ERNIE-Tutorial训练教程 | 定制化模型开发 |
| **结构化输出** | SGLang结构化生成文档 | JSON生成、代码生成 |


### 联系我们

如果您在使用过程中遇到问题或有任何建议，欢迎通过以下方式联系：

- **GitHub Issues**：在项目仓库中提交问题
- **微信**：G_Fuji
- **社区论坛**：参与相关技术讨论